In [1]:
# load libraries and set plot parameters
import numpy as np
import tables as pt

from sympy import *

from IPython.display import display, Math, Latex

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 75

plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "cm"
# plt.rcParams['text.latex.preamble'] = "\usepackage{subdepth}, \usepackage{type1cm}"

# The zero-lift angle of a finite wing

We call $\alpha_{0L}$ the zero-lift angle of a wing. If the free-stream velocity has an angle $\alpha_{\mathrm{W}}=\alpha_{0L}$ with respect to the wing root chord then the lift is zero: $C_L = 0$.

The zero-lift angle of attack is given by the following formula:

\begin{equation}
\label{eq:AlphaZeroLift}
\alpha_{0L} 
  = \frac{2}{S} \int_0^{b/2} 
    \left[\alpha_{0\ell}(y) - \epsilon_{\mathrm{g}}(y)\right]
    c(y) \, \mathrm{d}y
\end{equation}

where $y$ is the generic coordinate along the wing span, $\alpha_{0\ell}(y)$ is the zero-lift angle of attack of the local airfoil at section, $\epsilon_{\mathrm{g}}(y)$ is the local section incidence with respect to the wing root chord, and $c(y)$ is the local chord.

## Numerical example

Let us consider an example where we calculate (\ref{eq:AlphaZeroLift}) numerically. 

In [2]:
c_r = 1.0; c_t = 0.5
b = 7.5

Our wing has a root chord $c_{\mathrm{r}} = \,${{c_r}}$\,\text{m}$, a span $b = \,${{b}}$\,\text{m}$, and a tip chord $c_{\mathrm{t}} = \,${{c_t}}$\,\text{m}$.

In [3]:
S = (c_r + c_t)*b/2
taper_ratio = c_t/c_r
AR = b*b/S
print("Taper ratio: {0}\nWing area: {1} m^2\nAspect ratio: {2}".format(taper_ratio,S,AR))

Taper ratio: 0.5
Wing area: 5.625 m^2
Aspect ratio: 10.0


The reference wing area, given by the formula
\begin{equation}
\label{eq:WingSurface}
S=( c_{\mathrm{r}} + c_{\mathrm{t}} )\,\frac{1}{2} b
\end{equation}

becomes $S = \,${{S}}$\,\text{m}^2$.
The taper ratio

\begin{equation}
\label{eq:TaperRatio}
\lambda = \frac{ c_{\mathrm{t}} }{ c_{\mathrm{r}} }
\end{equation}

becomes $\lambda = \,${{taper_ratio}}.

In [4]:
A_c = 2*(c_t - c_r)/b; B_c = c_r;
print("A_c: {0}\nB_c: {1} m".format(A_c,B_c))

A_c: -0.13333333333333333
B_c: 1.0 m


The law of the chords

\begin{equation}
\label{eq:ChordsLaw}
c(y) = A_c \, y + B_c
\end{equation}

with 

\begin{equation}
\label{eq:ChordsLawAB}
A_c = \frac{2}{b} \left( c_{\mathrm{t}} - c_{\mathrm{r}} \right)
  \;\;,\qquad 
  B_c = c_{\mathrm{r}}
\end{equation}

becomes $c(y) = \,${{A_c}}$\,y + \,${{B_c}}$\,\text{m}$.



In [5]:
y = Symbol('y')
ltx_int = latex(integrate(A_c*y + B_c,y))
int_def = 2*integrate(A_c*y + B_c,(y,0,b/2))
print(ltx_int)
print(int_def)

- 0.0666666666666667 y^{2} + 1.0 y
5.62500000000000


In [6]:
display(
    Math(
        r'S = 2 \int_0^{'+str(b/2)+'} \left(' 
        + str(A_c) + 'y + ' + str(B_c)
        + r'\right)\,\text{d}y =' 
        + r'2\left(' + ltx_int + r'\right)\Bigr|_0^{'+str(b/2)+'}'
        + '=' + str(S) + r'\,\text{m}^2'
    ))

<IPython.core.display.Math object>

In [7]:
Latex(
    r'\begin{equation}'
    + r'S = 2 \int_0^{'+str(b/2)+'} \left(' 
    + str(A_c) + 'y + ' + str(B_c)
    + r'\right)\,\text{d}y =' 
    + r'2\left(' + ltx_int + r'\right)\Bigr|_0^{'+str(b/2)+'}'
    + '=' + str(S) + r'\,\text{m}^2'
    +r'\end{equation}'
    )

<IPython.core.display.Latex object>

# Python markdown

In [8]:
from IPython.display import Latex
aa = Latex('$\int_0^\infty a(x) dx$')

The equation {{aa}} computer an integral.

See 
`https://github.com/ipython-contrib/IPython-notebook-extensions/wiki/Home-4.x-%28Jupyter%29`

In [9]:
from __future__ import print_function
from jupyter_core.paths import jupyter_config_dir, jupyter_config_path
print(jupyter_config_dir())
print(jupyter_config_path())

C:\Users\jyota\.jupyter
['C:\\Users\\jyota\\.jupyter', 'C:\\Users\\jyota\\Anaconda3\\etc\\jupyter', 'C:\\ProgramData\\jupyter']


In [10]:
from __future__ import print_function
from jupyter_core.paths import jupyter_data_dir, jupyter_path
print(jupyter_data_dir())
print(jupyter_path())

C:\Users\jyota\AppData\Roaming\jupyter
['C:\\Users\\jyota\\AppData\\Roaming\\jupyter', 'C:\\Users\\jyota\\Anaconda3\\share\\jupyter', 'C:\\ProgramData\\jupyter']


In [11]:
import notebook
notebook.nbextensions.check_nbextension('usability/codefolding', user=True)
notebook.nbextensions.check_nbextension('usability/codefolding/main.js', user=True)

True

In [12]:
import notebook
E = notebook.nbextensions.EnableNBExtensionApp()
E.enable_nbextension('usability/codefolding/main')

In [13]:
a = 1.23

a = {{a}}